In [2]:
import pandas as pd
data = pd.read_csv('netflix_content.csv')
data.head()

,Title,Available Globally?,Release Date,Hours Viewed,Language Indicator,Content Type
0,The Night Agent: Season 1,Yes,2023-03-23,"81,21,00,000",English,Show
1,Ginny & Georgia: Season 2,Yes,2023-01-05,"66,51,00,000",English,Show
2,The Glory: Season 1 // 더 글로리: 시즌 1,Yes,2022-12-30,"62,28,00,000",Korean,Show
3,Wednesday: Season 1,Yes,2022-11-23,"50,77,00,000",English,Show
4,Queen Charlotte: A Bridgerton Story,Yes,2023-05-04,"50,30,00,000",English,Movie


In [6]:
# drop rows with missing titles or with duplicate titles
data.dropna(subset = ['Title'], inplace = True)
data.drop_duplicates(subset = ['Title'], inplace = True)

# Create simple content ID for TensorFlow embeddings
data['Content_ID'] = data.reset_index().index.astype('int32')

# encode language indicator and content type
data['Language_ID'] = data['Language Indicator'].astype('category').cat.codes
data['ContentType_ID'] = data['Content Type'].astype('category').cat.codes
data[['Content_ID', 'Title', 'Hours Viewed', 'Language_ID', 'ContentType_ID']].head()

,Content_ID,Title,Hours Viewed,Language_ID,ContentType_ID
0,0,The Night Agent: Season 1,812100000,0,1
1,1,Ginny & Georgia: Season 2,665100000,0,1
2,2,The Glory: Season 1 // 더 글로리: 시즌 1,622800000,3,1
3,3,Wednesday: Season 1,507700000,0,1
4,4,Queen Charlotte: A Bridgerton Story,503000000,0,0


In [ ]:

import tensorflow as tf
from tensorflow import layers, Model

num_content = data['Content_ID'].nunique()
num_language = data['Language_ID'].nunique()
num_type = data['ContentType_ID'].nunique()

content_input = layers.Input(shape=(1,), dtype=tf.int32, name='content_id')
language_input = layers.Input(shape=(1,), dtype=tf.int32, name='language_id')
type_input = layers.Input(shape=(1,), dtype=tf.int32, name='type_id')

content_embedding = layers.Embedding(input_dim=num_content+1, output_dim=32, name='content_embedding')(content_input)
language_embedding = layers.Embedding(input_dim=num_language+1, output_dim=4, name='language_embedding')(language_input)
type_embedding = layers.Embedding(input_dim=num_type+1, output_dim=4, name='type_embedding')(type_input)

content_vec = layers.Flatten()(content_embedding)
language_vec = layers.Flatten()(language_embedding)
type_vec = layers.Flatten()(type_embedding)

combined = layers.Concatenate()([content_vec, language_vec, type_vec])
x = layers.Dense(64, activation='relu')(combined)
x = layers.Dense(32, activation='relu')(x)
output = layers.Dense(num_content, activation='softmax')(x)

model = Model(inputs=[content_input, language_input, type_input], outputs=output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

